In [169]:
import pandas as pd
import numpy as np

In [185]:
# read in timeseries data
path = "W://Non-APS//CEC PIER//PIER Solar Water Heating//Publications//ACEEE//data//"

com = pd.read_csv(path + 'community4houshlds_new_3occ_each_cold_cz1_july_23_to_30.csv')
ind = pd.read_csv(path + 'individiual_new_3occ_cold_cz1_july_23_to_30.csv')

# rename "Unnamed: 0" to "Hour of Year"
com = com.rename(columns = {"Unnamed: 0" : "Hour of Year"})
ind = ind.rename(columns = {"Unnamed: 0" : "Hour of Year"})

# add a column with percentage unmet demand
com["Unmet Demand"] = (com["Tank Unmet Heat"]/com["Net Heat Demand"])
com["Unmet Demand"].fillna(0, inplace=True)
ind["Unmet Demand"] = (ind["Tank Unmet Heat"]/ind["Net Heat Demand"])
ind["Unmet Demand"].fillna(0, inplace=True)

In [178]:
com.head(15)

,Hour of Year,Net Heat Demand,Tank Heat Delivered,Tank Unmet Heat,Solar Heat Delivered To Tank,Distribution Heat Loss,Unmet Demand
0,4873,295.61,240.80,56.52,0.000000,1.707602,0.191198
1,4874,0.00,0.00,0.00,0.000000,0.000000,0.000000
2,4875,109.80,88.69,21.74,0.000000,0.628428,0.197996
3,4876,0.00,0.00,0.00,0.000000,0.000000,0.000000
4,4877,1570.94,1263.20,316.69,0.000000,8.947250,0.201593
5,4878,143.58,111.38,32.99,-0.000000,0.789126,0.229767
6,4879,658.78,508.38,154.11,-0.000000,3.706043,0.233932
7,4880,2170.61,1648.04,534.18,-0.000000,11.609540,0.246097
8,4881,7128.37,5152.91,2011.58,185.826127,36.120227,0.282194
9,4882,751.69,463.50,291.38,956.753588,3.190999,0.387633


In [179]:
# not needed for the plots, as plotly allows for ticks to be mapped (see below before the plot)
def append_day_of_week__based_on__hour_of_year(data, hour_of_year_label):
    """If the dataframe contains an hour of year column this
    method appends a column with a string day of week label.
    
    Assumes the year starts with hour 0.
    """
    # Day of the week
    data['Weekday Numerical'] = 1 + (data[hour_of_year_label]/24).astype(int) % 7
    data['Weekday'] = data['Weekday Numerical'].map(
        {1: 'Mon', 2: 'Tue', 3: 'Wed', 4: 'Thu', 5: 'Fri', 6: 'Sat', 7: 'Sun'})
    
    return data

In [180]:
# add weekday column
com = append_day_of_week__based_on__hour_of_year(com, "Hour of Year")
ind = append_day_of_week__based_on__hour_of_year(ind, "Hour of Year")

In [181]:
ind.head()

,Hour of Year,Net Heat Demand,Tank Heat Delivered,Tank Unmet Heat,Solar Heat Delivered To Tank,Distribution Heat Loss,Unmet Demand,Weekday Numerical,Weekday
0,4873,0.0,0.0,0.0,0.0,0,0,1,Mon
1,4874,0.0,0.0,0.0,0.0,0,0,1,Mon
2,4875,0.0,0.0,0.0,0.0,0,0,1,Mon
3,4876,0.0,0.0,0.0,0.0,0,0,1,Mon
4,4877,0.0,0.0,0.0,0.0,0,0,1,Mon


In [193]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# place individual system results on the left and community system on the right plots
# display demand and heat delivered from the solar tank in the first row of plots
# display percentage unmet demand in the second row 
# display solar heat fed to the tank in the third row

fig = make_subplots(rows=3, cols=2, shared_xaxes=True, shared_yaxes=True, \
                    vertical_spacing=0.03, horizontal_spacing=0.02)


fig.add_trace(
    go.Scatter(x=ind["Hour of Year"], y=ind["Net Heat Demand"], name = "Net Heat Demand, W",
              line = dict(color='royalblue', width=2)),
    row=1, col=1,
)

fig.add_trace(
    go.Scatter(x=ind["Hour of Year"], y=ind["Tank Heat Delivered"], name = "Tank Heat Delivered, W",
              line = dict(color='red', width=1, dash='dot')),
    row=1, col=1
)


fig.add_trace(
    go.Scatter(x=com["Hour of Year"], y=com["Net Heat Demand"], name = "Tank Heat Delivered, W",
              line = dict(color='royalblue', width=2)),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=com["Hour of Year"], y=com["Tank Heat Delivered"], name = "Net Heat Demand, W",
              line = dict(color='red', width=1, dash='dot')),
    row=1, col=2
)


fig.add_trace(
    go.Scatter(x=com["Hour of Year"], y=com["Tank Heat Delivered"], name = "Net Heat Demand, W",
              line = dict(color='royalblue', width=4, dash='dash')),
    row=1, col=2
)


fig.add_trace(
    go.Scatter(x=ind["Hour of Year"], y=100*ind["Unmet Demand"], name = "Percentage Unmet Demand, W",
              line = dict(color='royalblue', width=4, dash='dash')),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=ind["Hour of Year"], y=100*com["Unmet Demand"], name = "Percentage Unmet Demand, W",
              line = dict(color='royalblue', width=4, dash='dash')),
    row=2, col=2
)

fig.add_trace(
    go.Scatter(x=ind["Hour of Year"], y=ind["Solar Heat Delivered To Tank"], name = "Solar Heat Delivered To Tank, W",
              line = dict(color='royalblue', width=4, dash='dash')),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(x=com["Hour of Year"], y=com["Solar Heat Delivered To Tank"], name = "Solar Heat Delivered To Tank, W",
              line = dict(color='royalblue', width=4, dash='dash')),
    row=3, col=2
)


# Update xaxis properties
fig.update_xaxes(title_text="Individual Household", row=3, col=1,
        tickmode = 'array',
        tickvals = [4872 + 11, 4872 + 11 + 24, 4872 + 11 + 24*2, \
                    4872 + 11 + 24*3, 4872 + 11 + 24*4, 4872 + 11 + 24*5, 4872 + 11 + 24*6, 4872 + 11 + 24*7],
        ticktext = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    )

fig.update_xaxes(title_text="Community of Four Households", row=3, col=2,
        tickmode = 'array',
        tickvals = [4872 + 11, 4872 + 11 + 24, 4872 + 11 + 24*2, \
                    4872 + 11 + 24*3, 4872 + 11 + 24*4, 4872 + 11 + 24*5, 4872 + 11 + 24*6, 4872 + 11 + 24*7],
        ticktext = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    )

# Update yaxis properties
fig.update_yaxes(title_text="Heat Flow, W", range=[0, 10000], row=1, col=1)
fig.update_yaxes(title_text="Unmet Demand, %", range=[0, 100], row=2, col=1)
fig.update_yaxes(title_text="Heat Flow, W", range=[0, 10000], row=3, col=1)

fig.update_yaxes(title_text="", range=[0, 10000], row=1, col=2)
fig.update_yaxes(title_text="", range=[0, 100], row=2, col=2)
fig.update_yaxes(title_text="", range=[0, 10000], row=3, col=2)

# set size
fig.update_layout(height=700, width=1000,
                  title_text="")

fig.update_layout(showlegend=False)

fig.show()

In [55]:
(4875/24) % 7

0.125

In [139]:
ind["Net Heat Demand"]

0         0.00
1         0.00
2         0.00
3         0.00
4         0.00
        ...   
164       0.00
165      42.23
166    2111.48
167       0.00
168       0.00
Name: Net Heat Demand, Length: 169, dtype: float64